# Crawling

To crawl the page: https://isa.epfl.ch/imoniteur_ISAP/!gedpublicreports.htm?ww_i_reportmodel=66627699, we have used the following code to extract the course ww_x_MAT.
Run this code on the page, but first follow these steps:
1. select format "pour excel"
2. press the button "OK"
3. Wait that the page shows the courses
4. Inspect the table related to "2: Cliquez sur une des matières pour avoir les inscriptions" in this way we will attach the console to the frame selected.
5. Run the following code in the console.

```
    let tbody = document.getElementsByClassName("filtres")[0].children[3].children[0]
    let trs = tbody.children
    trs = Array.prototype.slice.call(trs)

    function extrapolateExcel(value, idx, array){
        return (value.children[0].children[0].getAttribute("onclick").toString())
    }
    let res = trs.map(extrapolateExcel)
    
    
    let saveData = (function () {
        let a = document.createElement("a");
        document.body.appendChild(a);
        a.style = "display: none";
        return function (data, fileName) {
            var json = JSON.stringify(data),
                blob = new Blob([json], {type: "octet/stream"}),
                url = window.URL.createObjectURL(blob);
            a.href = url;
            a.download = fileName;
            a.click();
            window.URL.revokeObjectURL(url);
        };
    }());
    
    saveData(res, "idCourses.txt")
```

6. Clean now the obtained file to just have the 'ww_x_MAT= idCourse'
7. Observing the requests sent by the server when we try to download the excel of a course we can see the url used to send a GET request (https://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.XLS?ww_x_MAT=405249038&ww_x_CLASSE=null&ww_i_reportModel=66627699&ww_x_MATIERE=&ww_x_PERIODE_ACAD=null&ww_i_reportModelXsl=66627727&ww_x_HIVERETE=null). 
8. We have therefore created a text file containing all the urls with the excel files that we want to download. 
9. Run the code below to download all the files.

In [1]:
from threading import Thread
import functools

def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print('error starting thread')
                raise je
            ret = res[0]
            if isinstance(ret, BaseException):
                raise ret
            return ret
        return wrapper
    return deco



In [2]:
import wget
from time import sleep

@timeout(timeout=10) # runned at first with a small timeout and after we have increased it to download some specific files
def download(url, i):
    wget.download(url, out='../../data/{}.xls'.format(i))
    
with open('./idCourses.txt') as f:
    for i, url in enumerate(f):
        try:
            sleep(1)
            download(url, i)
        except:
            print('{}: {}'.format(i, url))
            pass
            